<span style="color: rgb(74, 74, 74); font-family: -apple-system, system-ui, sans-serif; font-size: 14px; white-space: normal;">Infection rate of each location (reported cases vs population). Consider reported cases, not unknown/empty</span>

In [ ]:
/*SELECT continent, location, date, population, total_cases, 
       ROUND(((CAST(total_cases AS float) / CAST(population AS float)) * 100), 4) AS Infection_rate
FROM Covid_Infection..Covid_Death_Rate
WHERE total_cases IS NOT NULL 
ORDER BY location, 4;*/

--get the general infection rate for each countries
WITH total AS (
       SELECT location, population, AVG(total_cases) AS total_cases
       FROM Covid_Infection..Covid_Death_Rate
       WHERE total_cases IS NOT NULL
       GROUP BY location, population 
)
SELECT location, population, total_cases, 
       ROUND(((CAST(total_cases AS float) / CAST(population AS float)) * 100), 4) AS Infection_rate
FROM total
WHERE total_cases IS NOT NULL 
ORDER BY location, population

Top 5 country with highest infection rate

In [ ]:
SELECT TOP 5 location, population, MAX(total_cases) AS highest_case, 
       ROUND(MAX((CAST(total_cases AS float) / CAST(population AS float)) * 100), 4) AS Infection_rate
FROM Covid_Infection..Covid_Death_Rate
WHERE total_cases IS NOT NULL
GROUP BY location, population 
ORDER BY Infection_rate DESC


Infection rate for each continent

In [ ]:
--need to find avg cases for each country, then sum the result
/*
SELECT continent, AVG(total_cases) AS avgcase
FROM Covid_Infection..Covid_Death_Rate
WHERE total_cases IS NOT NULL
GROUP BY continent, location;
*/
WITH total AS (
    SELECT continent, AVG(total_cases) AS avgcase
    FROM Covid_Infection..Covid_Death_Rate
    WHERE total_cases IS NOT NULL
    GROUP BY continent, location
)
SELECT total.continent, SUM(dr.population) AS total_population, SUM(total.avgcase) AS continental_cases,
        ROUND(((CAST(SUM(total.avgcase) AS float) / CAST(SUM(dr.population) AS float)) * 100),4) AS Infection_rate
FROM Covid_Infection..Covid_Death_Rate dr
JOIN total
    ON dr.continent = total.continent
GROUP BY total.continent
ORDER BY total.continent;




Is there a corrolation between population density and infection rate? (join both table, need to group by location)

(Check calculation by find out how many country are there and if it match)

In [ ]:
--want to consider location where population density (in Covid Vaccination) is reported
/*
SELECT location, date, population_density
FROM Covid_Infection..Covid_Vaccination
WHERE population_density IS NOT NULL
ORDER BY population_density;
*/
--consider pop_density and infection rate
WITH infected_percent AS (
    SELECT location, population, AVG(total_cases) AS avg_case, 
           ROUND(((CAST(AVG(total_cases) AS float) / CAST(population AS float)) * 100), 4) AS infection_rate
    FROM Covid_Infection..Covid_Death_Rate
    WHERE total_cases IS NOT NULL
    GROUP BY location, population
)
SELECT va.location, va.population_density, infected_percent.population,
       infected_percent.avg_case, infected_percent.infection_rate
FROM Covid_Infection..Covid_Vaccination va
JOIN infected_percent
     ON va.location = infected_percent.location
WHERE va.population_density IS NOT NULL
GROUP BY va.location, va.population_density, infected_percent.location, 
         Infected_percent.population, infected_percent.avg_case, infected_percent.infection_rate
ORDER BY va.population_density DESC;




In [ ]:
--checking queries/calculation 
SELECT COUNT(DISTINCT(location)) AS total_amount_of_country
FROM Covid_Infection..Covid_Death_Rate
WHERE total_cases IS NOT NULL 

In [ ]:
--now calculate the correlation
    --create table with query finding
DROP TABLE IF EXISTS Infected_Rate
CREATE TABLE Infected_Rate (
    Location VARCHAR(50),
    Population VARCHAR(50),
    Population_Density FLOAT,
    Infection_Rate FLOAT
)
INSERT INTO Infected_Rate
SELECT va.location, infected_percent.population, va.population_density, infected_percent.Infection_rate
FROM Covid_Infection..Covid_Vaccination va, 
    (SELECT location, population, AVG(total_cases) AS avg_case, 
     ROUND(((CAST(AVG(total_cases) AS float) / CAST(population AS float)) * 100), 4) AS Infection_rate
     FROM Covid_Infection..Covid_Death_Rate
     WHERE total_cases IS NOT NULL
     GROUP BY location, population) infected_percent
WHERE va.location = infected_percent.location
AND va.population_density IS NOT NULL
GROUP BY va.location, va.population_density, infected_percent.location, 
         infected_percent.population, infected_percent.Infection_rate
ORDER BY va.population_density DESC;

--calculate mean, var(), std deviation
WITH Mean AS (
    SELECT Population_Density, Infection_Rate,
           AVG(Population_Density) OVER() AS density_mean, 
           AVG(Infection_Rate) OVER() AS infection_mean
    FROM Infected_Rate
),
Variance AS (
    SELECT AVG(POWER((Population_Density - density_mean), 2)) AS density_varr, 
           AVG(POWER((Infection_Rate - infection_mean), 2)) AS infection_varr
    FROM Mean
),
Std_Dev AS (
    SELECT STDEV(Population_Density) AS density_stdev, 
           STDEV(Infection_Rate) AS infection_stdev
    FROM Mean
),
Covariance As (
    SELECT AVG((Population_Density - density_mean) * (Infection_Rate - infection_mean)) AS cov
    FROM Mean
)
SELECT ROUND(cov / (density_stdev * infection_stdev), 4) AS Correlation 
FROM Std_Dev, Covariance;


Are countries with lower GDP more likely to be infected? (consider gdp and infection rate)

In [ ]:
--want to consider countries with reported gdp only
/*
SELECT location, gdp_per_capita
FROM Covid_Infection..Covid_Vaccination
WHERE gdp_per_capita IS NOT NULL
GROUP BY location, gdp_per_capita
ORDER BY gdp_per_capita;
*/

--consider gdp with infection rate
WITH infection_rate AS (
    SELECT location, population, AVG(total_cases) AS avgcase, 
           ROUND(((CAST(AVG(total_cases) AS float) / CAST(population AS float)) * 100), 4) AS Infection_rate
    FROM Covid_Infection..Covid_Death_Rate
    WHERE total_cases IS NOT NULL
    GROUP BY location, population
)
SELECT va.location, infection_rate.population, va.gdp_per_capita, infection_rate.Infection_rate
FROM Covid_Infection..Covid_Vaccination va
JOIN infection_rate
    ON va.location = infection_rate.location 
WHERE gdp_per_capita IS NOT NULL
GROUP BY va.location, infection_rate.population, va.gdp_per_capita, infection_rate.Infection_rate
ORDER BY infection_rate.population, va.gdp_per_capita;

In [ ]:
--find correlation
DROP TABLE IF EXISTS Infecting_Rate
CREATE TABLE Infecting_Rate (
    Location VARCHAR(50),
    Population VARCHAR(50),
    GDP_Per_Capita FLOAT,
    Infection_Rate FLOAT
)
INSERT INTO Infecting_Rate
SELECT va.location, infection_rate.population, va.gdp_per_capita, infection_rate.Infection_rate
FROM Covid_Infection..Covid_Vaccination va, 
     (SELECT location, population, AVG(total_cases) AS avgcase, 
           ROUND(((CAST(AVG(total_cases) AS float) / CAST(population AS float)) * 100), 4) AS Infection_rate
    FROM Covid_Infection..Covid_Death_Rate
    WHERE total_cases IS NOT NULL
    GROUP BY location, population) AS infection_rate
WHERE va.location = infection_rate.location 
AND gdp_per_capita IS NOT NULL
GROUP BY va.location, infection_rate.population, va.gdp_per_capita, infection_rate.Infection_rate
ORDER BY infection_rate.population, va.gdp_per_capita;
WITH Mean AS (
    SELECT GDP_Per_Capita, Infection_Rate,
           AVG(GDP_Per_Capita) OVER() AS density_mean, 
           AVG(Infection_Rate) OVER() AS infection_mean
    FROM Infecting_Rate
),
Variance AS (
    SELECT AVG(POWER((GDP_Per_Capita - density_mean), 2)) AS gdp_varr, 
           AVG(POWER((Infection_Rate - infection_mean), 2)) AS infection_varr
    FROM Mean
),
Std_Dev AS (
    SELECT STDEV(GDP_Per_Capita) AS gdp_stdev, 
           STDEV(Infection_Rate) AS infection_stdev
    FROM Mean
),
Covariance As (
    SELECT AVG((GDP_Per_Capita - density_mean) * (Infection_Rate - infection_mean)) AS cov
    FROM Mean
)
SELECT location, ROUND(cov / (gdp_stdev * infection_stdev), 4) AS Correlation 
FROM Infecting_Rate, Std_Dev, Covariance;

Percentage of hospitalization among infected cases for each location around the world (some location don't have record)

In [ ]:
SELECT continent, location, date, total_cases, hosp_patients,
       ROUND((CAST(hosp_patients AS float)/ CAST(total_cases AS float) * 100), 4) AS hospital_percent
FROM Covid_Infection..Covid_Death_Rate
WHERE hosp_patients IS NOT NULL 
ORDER BY 2, 3

Total death among infected cases for each location (calculating the death rate)

In [ ]:
--consider countries with reported death cases 
SELECT continent, location, total_cases, total_deaths,
       ROUND((CONVERT(float, total_deaths) / CONVERT(float, total_cases) * 100),4) AS death_percent
FROM Covid_Infection..Covid_Death_Rate
WHERE total_deaths IS NOT NULL 
AND total_cases IS NOT NULL 

Testing percent for each country based on population of the country (what is percentage of people go testing)

In [ ]:
--consider countries with reported test cases
SELECT va.location, de.population, va.total_tests,
       ROUND((CONVERT(float, va.total_tests) / CONVERT(float, de.population) * 100), 4) AS test_percent
FROM Covid_Infection..Covid_Vaccination va
JOIN Covid_Infection..Covid_Death_Rate de
    ON va.location = de.location 
    AND va.date = de.date 
WHERE total_tests IS NOT NULL
ORDER BY va.location

Fully vaccination rate for each continent 

\*For each country, it the same as people\_fully\_vaccinated\_per\_hundred\*

In [ ]:
--continent: sum population and fully_vac to find continental vac percent
WITH total AS(
    SELECT va.continent, SUM(de.population) AS sum_pop, SUM(va.people_fully_vaccinated) AS sum_vac
    FROM Covid_Infection..Covid_Vaccination va
    JOIN Covid_Infection..Covid_Death_Rate de
        ON va.location = de.location 
    AND va.date = de.date 
    WHERE people_fully_vaccinated IS NOT NULL
    GROUP BY va.continent, de.continent
)
SELECT continent, sum_pop, sum_vac,
       ROUND((CONVERT(float, sum_vac) / CONVERT(float, sum_pop) * 100), 4) AS vac_percent
FROM total
ORDER BY continent


Do countries with higher fully vaccination rate have lower death rate?

In [ ]:
--find the lowest death rate
/*
SELECT location, MIN(total_cases) AS lowest_cases, MIN(total_deaths) AS lowest_death,
       MIN(CONVERT(float, total_deaths) / CONVERT(float, total_cases) * 100) AS death_percent
FROM Covid_Infection..Covid_Death_Rate
WHERE total_deaths IS NOT NULL 
AND total_cases IS NOT NULL 
GROUP BY location, total_cases
ORDER BY death_percent;

--find highest fully vaccination rate
SELECT location, MAX(people_fully_vaccinated_per_hundred) AS Highest_vaccination
FROM Covid_Infection..Covid_Vaccination
GROUP BY location 
ORDER BY Highest_vaccination DESC;
*/
--combine the two tables for reference
WITH low_death_rate AS (
   SELECT location, MIN(total_cases) AS lowest_cases, MIN(total_deaths) AS lowest_death,
       ROUND(MIN(CONVERT(float, total_deaths) / CONVERT(float, total_cases) * 100), 4) AS death_percent
   FROM Covid_Infection..Covid_Death_Rate
   WHERE total_deaths IS NOT NULL 
   AND total_cases IS NOT NULL 
   GROUP BY location
)
SELECT va.location, low_death_rate.lowest_cases, 
       va.people_fully_vaccinated_per_hundred, 
       low_death_rate.death_percent
FROM Covid_Infection..Covid_Vaccination va 
JOIN low_death_rate
    ON va.location = low_death_rate.location
ORDER BY va.people_fully_vaccinated_per_hundred DESC


In [ ]:
--find the correlation between the two
DROP TABLE IF EXISTS Death_Rate
CREATE TABLE Death_Rate (
    Location VARCHAR(50),
    Population VARCHAR(50),
    Fully_Vacc_Rates FLOAT,
    Death_Rates FLOAT
)
INSERT INTO Death_Rate
SELECT va.location, death_percent.population, va.people_fully_vaccinated, death_percent.Death_rate
FROM Covid_Infection..Covid_Vaccination va, 
    (SELECT location, population, AVG(total_deaths) AS avg_case, 
     ROUND(((CAST(AVG(total_deaths) AS float) / CAST(population AS float)) * 100), 4) AS Death_rate
     FROM Covid_Infection..Covid_Death_Rate
     WHERE total_deaths IS NOT NULL
     GROUP BY location, population) death_percent
WHERE va.location = death_percent.location
AND va.people_fully_vaccinated IS NOT NULL
GROUP BY va.location, va.people_fully_vaccinated, death_percent.location, 
         death_percent.population, death_percent.Death_rate
ORDER BY va.people_fully_vaccinated DESC;

--calculate mean, var(), std deviation
WITH Mean AS (
    SELECT Fully_Vacc_Rates, Death_Rates,
           AVG(Fully_Vacc_Rates) OVER() AS vacc_mean, 
           AVG(Death_Rates) OVER() AS death_mean
    FROM Death_Rate
),
Variance AS (
    SELECT AVG(POWER((Fully_Vacc_Rates - vacc_mean), 2)) AS vacc_varr, 
           AVG(POWER((Death_Rates - death_mean), 2)) AS death_varr
    FROM Mean
),
Std_Dev AS (
    SELECT STDEV(Fully_Vacc_Rates) AS vac_stdev, 
           STDEV(Death_Rates) AS death_stdev
    FROM Mean
),
Covariance As (
    SELECT AVG((Fully_Vacc_Rates - vacc_mean) * (Death_Rates - death_mean)) AS cov
    FROM Mean
)
SELECT ROUND(cov / (vac_stdev * death_stdev), 4) AS Correlation 
FROM Std_Dev, Covariance;


World stats: total cases, vaccination rate and death rate

In [ ]:
--join the tables
SELECT SUM(de.total_cases) AS global_cases, 
       SUM(de.total_deaths) AS global_deaths,
       SUM(va.people_fully_vaccinated) AS global_fully_vaccinated
FROM Covid_Infection..Covid_Death_Rate de 
JOIN Covid_Infection..Covid_Vaccination va
    ON de.location = va.location 
    AND de.date = va.date 